In [22]:
import blaze as bz
import numpy as np
import pandas as pd
import json
from datashape import dshape

from zipline.data import bundles
from zipline.utils.calendars import get_calendar

from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.pipeline.data import DataSet
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.filters import StaticAssets
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.pipeline.loaders.blaze import BlazeLoader, from_blaze

In [23]:
from alphatools.research import loaders, blaze_loader
from alphatools.research import run_pipeline

In [25]:
with open('../alphatools/data/factory/data_sources.json') as f:
    data_sources = json.load(f)

In [26]:
Factory = {}

for source in data_sources.keys():
    loc = data_sources[source]['url']
    shape = dshape(data_sources[source]['schema'])

    expr = bz.data(
    '../alphatools/data/factory/' + loc,
    dshape=shape
    )
    
    # create the DataSet
    ds = from_blaze(
        expr,
        no_deltas_rule='ignore',
        no_checkpoints_rule='ignore',
        loader=blaze_loader,
        missing_values={'index':-1}
    )
    Factory = {source: ds}

In [27]:
Factory

{u'sample': <DataSet: 'BlazeDataSet_1'>}

In [16]:
assets = bundle_data.asset_finder.lookup_symbols(['A', 'AAL'], as_of_date=None)

In [28]:
p = Pipeline(
    columns={
        'col_A': Factory['sample'].value.latest,
    },
    screen=StaticAssets(assets)
)

In [29]:
df = run_pipeline(
    p,
    pd.Timestamp('2016-01-05', tz='utc'),
    pd.Timestamp('2018-01-04', tz='utc')
)

In [30]:
df.head()

col_A
2016-01-05 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-06 00:00:00+00:00 Equity(0 [A])    0.543405
                          Equity(2 [AAL])  0.424518
2016-01-07 00:00:00+00:00 Equity(0 [A])    0.543405